# Import

In [38]:
import numpy as np
import pandas as pd
import datetime

import sys, os
sys.path.insert(0, os.path.abspath('scripts'))
import rus_table_parser

# Data parsing

In [39]:
DAY = datetime.datetime.now().strftime("%d").lstrip('0')
MONTH = datetime.datetime.now().strftime("%m").lstrip('0')
# DAY = 23
# MONTH = 5

# Save of "https://coronavirus-monitor.ru/coronavirus-v-rossii/" page
path = f'html_data/{DAY}-{MONTH}/Коронавирус в России. Онлайн карта распространения коронавируса в России..html'
parse_df = rus_table_parser.run_parsing(path)
parse_df

,Confirmed,Date,Deaths,Recovered,Region/City
0,161397.0,2020-05-23,1934.0,47413.0,Москва
1,32653.0,2020-05-23,324.0,5163.0,Московская область
2,12955.0,2020-05-23,113.0,3124.0,Санкт-Петербург
3,7826.0,2020-05-23,60.0,2257.0,Нижегородская область
4,3982.0,2020-05-23,70.0,3002.0,Республика Дагестан
...,...,...,...,...,...
80,154.0,2020-05-23,2.0,113.0,Севастополь
81,92.0,2020-05-23,0.0,46.0,Республика Алтай
82,66.0,2020-05-23,0.0,36.0,Сахалинская область
83,62.0,2020-05-23,0.0,10.0,Ненецкий автономный округ


# Preparing parsed data

In [40]:
file_name = 'release/covid19-russia-cases.csv'
rus_df = pd.read_csv(file_name)
rus_df.tail()

,Date,Region/City,Region/City-Eng,Region_ID,Day-Confirmed,Day-Deaths,Day-Recovered,Confirmed,Deaths,Recovered
4156,2020-05-22,Ханты-Мансийский АО,Khanty-Mansiysk,86.0,80.0,1.0,18.0,1510.0,13.0,520.0
4157,2020-05-22,Челябинская область,Chelyabinsk region,74.0,80.0,0.0,29.0,2132.0,11.0,754.0
4158,2020-05-22,Чеченская Республика,Chechen Republic,95.0,20.0,0.0,8.0,1046.0,11.0,643.0
4159,2020-05-22,Ямало-Ненецкий АО,Yamalo-Nenets Autonomous Okrug,89.0,87.0,0.0,74.0,1989.0,5.0,398.0
4160,2020-05-22,Ярославская область,Yaroslavl region,76.0,103.0,0.0,24.0,2309.0,13.0,444.0


In [41]:
# Create day-columns
parse_df['Day-Confirmed'] = 0
parse_df['Day-Deaths'] = 0
parse_df['Day-Recovered'] = 0

In [42]:
# Strip text data
rus_df['Region/City'] = rus_df['Region/City'].astype('str').str.strip('\u200b')
parse_df['Region/City'] = parse_df['Region/City'].astype('str').str.strip('\u200b')

In [43]:
# Rename regions
rename_dict = {
    'Татарстан': 'Республика Татарстан',
    'Башкортостан': 'Республика Башкортостан',
    'Чувашская Республика': 'Республика Чувашия',
    'Камчатский край' : 'Камчатский край',
    'Ямало-Ненецкий автономный округ' : 'Ямало-Ненецкий АО',
    'Республика Северная Осетия — Алания' : 'Республика Северная Осетия - Алания',
    'Республика Алтай' : 'Республика Алтай'
}

def rename(row):
    name = row['Region/City'][0]
    return pd.Series(rename_dict[name] if name in rename_dict else name)

parse_df['Region/City'] = parse_df.reset_index().groupby('index').apply(rename).reset_index(drop=True)

In [44]:
# Minus a day from now if needed
parse_df['Date'] = pd.to_datetime(parse_df['Date'])
# parse_df['Date'] = parse_df['Date'] - pd.Timedelta(days=1)

In [45]:
# Fit day-columns
def upd(row):
    reg = row['Region/City']
    
    row['Day-Confirmed'] = row['Confirmed'] - rus_df[rus_df['Region/City'] == reg]['Confirmed'].max()
    row['Day-Deaths']    = row['Deaths'] - rus_df[rus_df['Region/City'] == reg]['Deaths'].max()
    row['Day-Recovered'] = row['Recovered'] - rus_df[rus_df['Region/City'] == reg]['Recovered'].max()
    
    row['Day-Confirmed'] = row['Confirmed'] if np.isnan(row['Day-Confirmed']) else row['Day-Confirmed']
    row['Day-Deaths']    = row['Deaths'] if np.isnan(row['Day-Deaths']) else row['Day-Deaths']
    row['Day-Recovered'] = row['Recovered'] if np.isnan(row['Day-Recovered']) else row['Day-Recovered']
    
    return row.drop('Region/City')

parse_df = parse_df.groupby('Region/City').apply(lambda df: upd(df.iloc[0])).reset_index()

In [46]:
# Check for missed regions
parse_df[np.isnan(parse_df['Day-Confirmed'])]

,Region/City,Confirmed,Date,Deaths,Recovered,Day-Confirmed,Day-Deaths,Day-Recovered


# Data checking

In [47]:
print('Russia Confirmed:', parse_df.groupby('Region/City')['Confirmed'].max().sum(),
      'Day-Confirmed:', parse_df.groupby('Region/City')['Day-Confirmed'].sum().sum())
print('Russia Deaths:', parse_df.groupby('Region/City')['Deaths'].max().sum(),
      'Day-Deaths:', parse_df.groupby('Region/City')['Day-Deaths'].sum().sum())
print('Russia Recovered:', parse_df.groupby('Region/City')['Recovered'].max().sum(),
      'Day-Recovered:', parse_df.groupby('Region/City')['Day-Recovered'].sum().sum())

# rus_regs = rus_sum.groupby('Region/City')['Recovered'].max().reset_index()
# rus_regs['Recovered-ByDay'] = rus_sum.groupby('Region/City')['Day-Recovered'].sum().reset_index(drop=True)
# rus_regs.loc[rus_regs['Recovered'] != rus_regs['Recovered-ByDay']]
# # rus_regs.to_csv('rus_regs.csv')

Russia Confirmed: 336147.0 Day-Confirmed: 9431.0
Russia Deaths: 3398.0 Day-Deaths: 129.0
Russia Recovered: 108097.0 Day-Recovered: 8133.0


# Data saving

In [48]:
# Save full snapshot of current day
parse_df.to_csv(f'covid19-russia-cases-upd-full.csv', index=False)

In [49]:
# Filter regions without changes
parse_df = parse_df[(parse_df['Day-Confirmed'] != 0) |
                (parse_df['Day-Deaths'] != 0) |
                (parse_df['Day-Recovered'] != 0)]

In [50]:
# Save parsed data for future use
parse_df.to_csv(f'old_data/covid19-russia-cases-upd{DAY}-{MONTH}.csv', index=False)
parse_df.to_csv('covid19-russia-cases-upd.csv', index=False)